In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [10]:
df = pd.read_csv("../data/raw/raw.csv")
df.head(5)

/tmp/ipykernel_44686/845610186.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/raw.csv")


,Номер события,Номер животного,Ушная бирка животного,Пол,Номер лактации,Результат отела,Легкость отела,Дата рождения,Дней в сухостое предыдущей лактации,Дней стельности при событии,Номер группы животного,Предыдущий номер группы животного,Событие,Дни доения при событии,Дата события,Примечание события
0,0,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,РАСЧКОП,336,28.03.2019,КАН
1,1,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,КОГЛАВАК
2,2,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,ЛЕПТО
3,3,291,530073354,F,6,MA,1,30.07.2009,68,0,23,23,ПЕРЕВОД,448,08.04.2019,F001T023
4,4,291,530073354,F,6,MA,1,30.07.2009,68,0,23,23,ПРОДАНА,458,18.04.2019,ПРОДАНА ; Прочее ; УВЗ8


# Предобработка

In [11]:
col_mapper = {
    'Номер события'                      : 'event_id',
    'Пол'                                : 'sex',
    'Номер животного'                    : 'id',
    'Номер лактации'                     : 'lactation_number',
    'Результат отела'                    : 'birth_result',
    'Легкость отела'                     : 'birth_difficult',
    'Дата рождения'                      : 'birthday',
    'Дней в сухостое предыдущей лактации': 'days_no_milking',
    'Дней стельности при событии'        : 'days_pregnant',
    'Номер группы животного'             : 'group_id',
    'Предыдущий номер группы животного'  : 'prev_group_id',
    'Событие'                            : 'event',
    'Дни доения при событии'             : 'days_milking',
    'Дата события'                       : 'date',
    'Примечание события'                 : 'remark'
}

df.drop([col for col in df.columns if col not in col_mapper], axis=1, inplace=True)
df.columns = [col_mapper[col] for col in df.columns]
df.head()

,event_id,id,sex,lactation_number,birth_result,birth_difficult,birthday,days_no_milking,days_pregnant,group_id,prev_group_id,event,days_milking,date,remark
0,0,146,F,6,FA,1,29.03.2010,62,277,18,18,РАСЧКОП,336,28.03.2019,КАН
1,1,146,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,КОГЛАВАК
2,2,146,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,ЛЕПТО
3,3,291,F,6,MA,1,30.07.2009,68,0,23,23,ПЕРЕВОД,448,08.04.2019,F001T023
4,4,291,F,6,MA,1,30.07.2009,68,0,23,23,ПРОДАНА,458,18.04.2019,ПРОДАНА ; Прочее ; УВЗ8


## Проверка на пустые значения

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592307 entries, 0 to 592306
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   event_id          592307 non-null  int64 
 1   id                592307 non-null  int64 
 2   sex               592307 non-null  object
 3   lactation_number  592307 non-null  int64 
 4   birth_result      331357 non-null  object
 5   birth_difficult   592307 non-null  int64 
 6   birthday          592307 non-null  object
 7   days_no_milking   592307 non-null  int64 
 8   days_pregnant     592307 non-null  int64 
 9   group_id          592307 non-null  int64 
 10  prev_group_id     592307 non-null  int64 
 11  event             592307 non-null  object
 12  days_milking      592307 non-null  int64 
 13  date              592307 non-null  object
 14  remark            592307 non-null  object
dtypes: int64(9), object(6)
memory usage: 67.8+ MB


В childs ставится NaN если ещё не было ребенка или животное мужского пола.
В remark стоит '-' если примечание не требуется

## Генерация каунтеров

### Пол и дети

In [13]:
df['sex'] = df['sex'].apply(lambda x: 1 if x =='F' else 0)

In [27]:
df['childs_fa'] = df['birth_result'].apply(lambda x: str(x).count('FA'))
df['childs_ma'] = df['birth_result'].apply(lambda x: str(x).count('MA'))
df['childs_fd'] = df['birth_result'].apply(lambda x: str(x).count('FD'))
df['childs_md'] = df['birth_result'].apply(lambda x: str(x).count('MD'))

df['childs_m'] = df['birth_result'].apply(lambda x: str(x).count('M'))
df['childs_f'] = df['birth_result'].apply(lambda x: str(x).count('F'))
df['childs_d'] = df['birth_result'].apply(lambda x: str(x).count('D'))
df['childs_a'] = df['birth_result'].apply(lambda x: str(x).count('A'))


In [80]:
df=df.drop(columns=['birth_result'])
df.dtypes

event_id             int64
id                   int64
sex                  int64
lactation_number     int64
birth_difficult      int64
birthday            object
days_no_milking      int64
days_pregnant        int64
group_id             int64
prev_group_id        int64
event               object
days_milking         int64
date                object
info                object
dtype: object

**TODO** check

### Дата рождения

In [14]:
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df['birthday']  = pd.to_datetime(df['birthday'], infer_datetime_format=True)
df['age']  = ((df['date'] - df['birthday']) / np.timedelta64(1, 'D')).astype(int)

In [15]:
df=df.drop(columns=['birthday'])
df.dtypes

event_id                     int64
id                           int64
sex                          int64
lactation_number             int64
birth_result                object
birth_difficult              int64
days_no_milking              int64
days_pregnant                int64
group_id                     int64
prev_group_id                int64
event                       object
days_milking                 int64
date                datetime64[ns]
remark                      object
age                          int64
dtype: object

### Дата события

In [24]:
month=['jan','feb','mar','apr','may','jun', 'jul', 'aug', 'sep', 'oct', 'nov',' dec']
for m in range (12):
    df[month[m]]=df['date'].apply(lambda x: 1 if x.month == m + 1 else 0)
df.head()

,event_id,id,sex,lactation_number,birth_result,birth_difficult,days_no_milking,days_pregnant,group_id,prev_group_id,...,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,0,146,1,6,FA,1,62,277,18,18,...,1,0,0,0,0,0,0,0,0,0
1,1,146,1,6,FA,1,62,277,18,18,...,0,1,0,0,0,0,0,0,0,0
2,2,146,1,6,FA,1,62,277,18,18,...,0,1,0,0,0,0,0,0,0,0
3,3,291,1,6,MA,1,68,0,23,23,...,0,1,0,0,0,0,0,0,0,0
4,4,291,1,6,MA,1,68,0,23,23,...,0,1,0,0,0,0,0,0,0,0


In [25]:
for year in (2020, 2021):
    for month in range(12):
        print(df[(df['event'] == EventType.MAST) & (df['date'].dt.year == year) & (df['date'].dt.month == month+1)]['id'].count())

340
335
300
227
188
214
200
210
214
214
186
167
168
155
232
161
197
197
236
254
234
179
156
96


In [ ]:
df[df['event']]

## Обработка событий

In [16]:
import re
from enum import IntEnum

class EventType(IntEnum):
    OTHER   = 0
    #-------------
    ABORT   = 101  # 
    TOSCM   = 102  #
    NULSCM  = 103  #
    DNB     = 104  #
    BRED    = 105  #
    FRESH   = 106  #
    PREG    = 107  #
    PREGBEF = 108  #
    DRY     = 109  #
    DRY2    = 110 #
    OPEN    = 111 #
    #--------------
    VAC     = 201 #
    VACVIR  = 202 #
    FOOTRIM = 203 #
    HEALTH  = 204 #
    BROKE   = 205 #
    POT     = 206 #
    ILLMISC = 207 #
    LAME    = 208 #
    KETOS   = 209 #
    MAST    = 210 #
    METR    = 211 #
    PARES   = 212 #
    RP      = 213 #
    #-------------
    WEIGHT  = 301 #
    DEAD    = 302 #
    MOVE    = 303 #
    SOLD    = 304 #
    
    def __str__(self):
        return self.name
    
    @staticmethod
    def from_str(label):        
        global EventSynonims
        return EventSynonims[label]

EventNames = {
    EventType.ABORT   : ["ABORT", "АБОРТ"],  
    EventType.TOSCM   : ["TOSCM", "НА_СХЕМУ"], 
    EventType.NULSCM  : ["NULSCM","СО_СХЕМЫ"],
    EventType.DNB     : ["DNB", "НЕОСЕМ"],
    EventType.BRED    : ["BRED", "ОСЕМЕН"], 
    EventType.FRESH   : ["FRESH", "ОТЕЛ"],  
    EventType.PREG    : ["PREG", "СТЕЛН"], 
    EventType.PREGBEF : ["PREGBEF", "СТЕЛНДО"],
    EventType.DRY     : ["DRY", "СУХОСТ"],
    EventType.DRY2    : ["DRY2", "СУХ2"],
    EventType.OPEN    : ["OPEN", "ЯЛОВАЯ"],
    #-------
    EventType.WEIGHT  : ["WEIGHT", "ВЕС"],
    EventType.DEAD    : ["DEAD", "ПАЛА"],
    EventType.MOVE    : ["MOVE", "ПЕРЕВОД"],
    EventType.SOLD    : ["SOLD", "ПРОДАНА"],
    #-------
    EventType.VAC     : ["VAC", "ВАКЦИН"],
    EventType.VACVIR  : ["VACVIR", "ВАКВИРУС"],
    EventType.FOOTRIM : ["FOOTRIM", "РАСЧКОП"],
    EventType.HEALTH  : ["HEALTH", "WELL", "ЗДОРОВА"],
    EventType.BROKE   : ["BROKE", "ДЕФЕКТ"],
    EventType.POT     : ["POT", "ПРОФОТ"],
    EventType.ILLMISC : ["ILLMISC", "БОЛЕЗНЬ"],
    EventType.LAME    : ["LAME", "ХРОМОТА"],
    EventType.KETOS   : ["KETOS", "КЕТОЗ"],
    EventType.MAST    : ["MAST", "МАСТИТ"],
    EventType.METR    : ["METR", "МЕТРИТ"],
    EventType.PARES   : ["PARES", "ПАРЕЗ"],
    EventType.RP      : ["RP", "ПОСЛЕД"],
}

EventSynonims = dict()
for event in EventNames.keys():
    for label in EventNames[event]:
        EventSynonims[label] = event
    

In [17]:
df['event']      = df['event'].apply(lambda x: EventType.from_str(x))
df['event_name'] = df['event'].apply(lambda x: EventNames[x][0])
df.head()

,event_id,id,sex,lactation_number,birth_result,birth_difficult,days_no_milking,days_pregnant,group_id,prev_group_id,event,days_milking,date,remark,age,event_name
0,0,146,1,6,FA,1,62,277,18,18,203,336,2019-03-28,КАН,3286,FOOTRIM
1,1,146,1,6,FA,1,62,277,18,18,201,349,2019-04-10,КОГЛАВАК,3299,VAC
2,2,146,1,6,FA,1,62,277,18,18,201,349,2019-04-10,ЛЕПТО,3299,VAC
3,3,291,1,6,MA,1,68,0,23,23,303,448,2019-04-08,F001T023,3539,MOVE
4,4,291,1,6,MA,1,68,0,23,23,304,458,2019-04-18,ПРОДАНА ; Прочее ; УВЗ8,3549,SOLD


### Примечания к событиям
События делим на три группы:
1. События без примечаний
2. События, примечания к которым обрабатываются однообразно (например примечания --- номер протокола или id семени)
3. События, примечания к которым обрабытваются сложным образом.

Чтобы выбрать событие третьего типа, надо выбрать правило которым его надо обработать, а затем собственно произвести обработку.

In [18]:
for event in EventType:
    print(event, df[df['event'] == event]['remark'].unique().tolist())
    print('---------------------------------------------------------------------------')

OTHER []
---------------------------------------------------------------------------
ABORT ['189 ДНИ', '146 ДНИ', '205 ДНИ', '94 ДНИ', '95 ДНИ', '217 ДНИ', '93 ДНИ', '167 ДНИ', '182 ДНИ', '141 ДНИ', '184 ДНИ', '51 ДНИ', '208 ДНИ', '58 ДНИ', '69 ДНИ', '155 ДНИ', '46 ДНИ', '206 ДНИ', '65 ДНИ', '200 ДНИ', '90 ДНИ', '33 ДНИ', '194 ДНИ', '56 ДНИ', '68 ДНИ', '67 ДНИ', '96 ДНИ', '59 ДНИ', '89 ДНИ', '91 ДНИ', '48 ДНИ', '55 ДНИ', '188 ДНИ', '181 ДНИ', '136 ДНИ', '214 ДНИ', '196 ДНИ', '170 ДНИ', '61 ДНИ', '104 ДНИ', '120 ДНИ', '101 ДНИ', '62 ДНИ', '76 ДНИ', '84 ДНИ', '85 ДНИ', '219 ДНИ', '229 ДНИ', '192 ДНИ', '78 ДНИ', '39 ДНИ', '60 ДНИ', '82 ДНИ', '105 ДНИ', '103 ДНИ', '216 ДНИ', '165 ДНИ', '134 ДНИ', '44 ДНИ', '110 ДНИ', '169 ДНИ', '40 ДНИ', '53 ДНИ', '215 ДНИ', '151 ДНИ', '176 ДНИ', '80 ДНИ', '108 ДНИ', '98 ДНИ', '88 ДНИ', '77 ДНИ', '198 ДНИ', '73 ДНИ', '228 ДНИ', '221 ДНИ', '140 ДНИ', '125 ДНИ', '145 ДНИ', '195 ДНИ', '123 ДНИ', '161 ДНИ', '171 ДНИ', '74 ДНИ', '47 ДНИ', '202 ДНИ', '257 ДНИ', 

Некоторые замечания:
1. В примечаниях к событиям категории OPEN встречается 'БПАК' что судя по всему является опечаткой от 'БРАК'
2. В примечаниях к событиям категории OPEN встречается 'БЖТ' и 'БЕЗ ЖТ' и вероятно это одно и тоже
3. В примечаниях к событиям категории SOLD причина продажи и код причины продажи не находятся во взаимнооднозначном соответствии
4. Примечания к событиям LAME и MAST не единобразны
5. В примечании к событии ILLMISC есть протокол БУР1_?

**Что делаем:**

Фиксим пункты 1 и 2, в примечаниях к событиям SOLD и DEAD оставляем только причину продажи/смерти (без кода). 

Примечания к событиям MAST и LAME приводим к виду {Код протокола}_{пораженные соски/ноги}

In [19]:

def spliter(s):
    t = s.split(';')
    if len(t) == 1:
        return 'Прочее'
    else:
        return t[1].strip().rstrip()

def get_protocol_id(s):
    return s[0:s.find('_'):]

def get_protocol_num(s):
    m = []
    t = s[s.find('_') + 1::]
    for i in range(len(t)):
        if t[i] == '_':
            for j in range(int(t[i - 1])+1, int(t[i + 1])):
                m.append(j)
        elif t[i].isdigit():
            m.append(int(t[i]))
    return m

df['remark'] = df.apply(lambda row: spliter(row.remark) if row.event == EventType.SOLD else row.remark, axis=1)
df['remark'] = df.apply(lambda row: spliter(row.remark) if row.event == EventType.DEAD else row.remark, axis=1)
df['remark'] = df.apply(lambda row: get_protocol_id(row.remark) + '_' + ''.join(list(map(str, get_protocol_num(row.remark)))) if (row.event == EventType.LAME or row.event == EventType.MAST) else row.remark, axis=1)

#### Механика парсинга:
К каждому событию составляется набор правил:

Правило это функция + регулярное выражение. Если примечание удовлетворяет регулярке, то его содержимое отправляется в функцию, а выход функции записывается в каунтер {ИМЯ СОБЫТИЯ}_{НОМЕР ПРАВИЛА}

In [20]:
df.head()

,event_id,id,sex,lactation_number,birth_result,birth_difficult,days_no_milking,days_pregnant,group_id,prev_group_id,event,days_milking,date,remark,age,event_name
0,0,146,1,6,FA,1,62,277,18,18,203,336,2019-03-28,КАН,3286,FOOTRIM
1,1,146,1,6,FA,1,62,277,18,18,201,349,2019-04-10,КОГЛАВАК,3299,VAC
2,2,146,1,6,FA,1,62,277,18,18,201,349,2019-04-10,ЛЕПТО,3299,VAC
3,3,291,1,6,MA,1,68,0,23,23,303,448,2019-04-08,F001T023,3539,MOVE
4,4,291,1,6,MA,1,68,0,23,23,304,458,2019-04-18,Прочее,3549,SOLD


In [32]:
class EventParsingRule:
    def __init__(self, pattern, parser):
        self.raw = pattern
        self.parse = parser
        self.pattern = re.compile(pattern)
    
        
RuleBin  = EventParsingRule('.*', lambda x: 1)
RuleNum  = EventParsingRule('[0-9]*', lambda x: int(x)) 
RuleDays = EventParsingRule('[0-9]* ((ДНИ)|(DAYS))', lambda x: int(x[0:-4]) if re.match('[0-9]* (ДНИ)', x) else int(x[0:-5]))

def satisfy(s, rules):
    for rule in rules:
        if rule.pattern.match(s):
            return True
    return False

def generate_unique_rules(event):
    global df
    patterns = df[df['event'] == event]['remark'].unique().tolist()
    rules = []
    for p in patterns:
        rules.append(EventParsingRule(p, lambda x: 1))
    return rules

def generate_combined_rules(specific_rules, event):
    global df
    rules = specific_rules.copy()
    patterns = df[df['event'] == event].loc[df['remark'].apply(lambda s: satisfy(s, specific_rules)) == False]['remark'].unique().tolist()
    for p in patterns:
        rules.append(EventParsingRule(p, lambda x: 1))
    return rules


def generate_protocol_rules(event):
    global df
    protocols = set(map(get_protocol_id, df[df['event'] == event]['remark'].unique().tolist()))
    rules = []
    for p in protocols:
        for i in range(1, 5):
            rules.append(EventParsingRule(p + '_' + f'(([0-9]+{i})|{i})[0-9]*', lambda x: 1))
    return rules

df = df[df['remark'] != '25.07.2022']
RemarkABORT = ['(БЖТ)|(БЕЗ ЖТ)', '(ЖТ)|(ЖТ,)', '(ДЛ СХЕМА)|(ДЛ СХ)|(ДС)']
RemarkDNB   = ['(ЗООТЕХ)|(ЗООТ)|(ЗООТ3)|(ЗООБРАК)|(ЗОOТЕХ)', '(СУСТАВ)|(СУСТАВЫ)|(СУСТ_ВЫМ)', '(ГИНЕКОЛ)|(ГЕНИКОЛ)|(ГЕНЕКОЛ)', '(АТП)|(АТ)', '(AM)|(АМ)|(АВС)|(АБСМАТКИ)|(АБС МАТК)|(МАТКА)|(Б/МАТКИ)', '(БРАК МОЛ)|(МОЛОКО)', '(ЖИРНАЯ)|(ЖИРН)', '(АБС)|(АБСЦЕСС)|(БРАК АБС)|(ABC)|(АБЦ)|(АБЦЕС)|(АБЦ ОПУХ)|(АБСМАТКА)', '(АГАЛАКТ)|(АГАЛАКТИ)', '(МАСТИТ)|(МАСТ)', '(ТУГОДОЙ)|(НАДОЙ)', '(ТРОЦНОГИ)|(ВЫМЯТРОЦ)|(НОГИТРОЦ)|(ТРОЦ)|(БРАКТРОЦ)', '(БРАК)|(БРАКНОГИ)', '(РОСТ)|(ТОНК СОС)|(НОГИ ТОЩ)|(ХУДАЯ)|(ТОЩ)|(НЕДОРОСТ)|(МЕЛКАЯ)|(ТОЩ СУСТ)|(ТОЩЬ)|(РАЗВИТИЕ)|(ВЕС)|(УЗКИЙТАЗ)', '(СЛЕПАЯ)|(ГЛАЗА)|(БЕЗГЛАЗА)', '(БЕЗМАТКИ)|(БЕЗМЕТКИ)']
RemarkOPEN  = ['(ФК)|(ФК, СТЕЛ)','(ЛК)|(ЛКЭ)','(ЖТ ПОЛИК)|(ЖТ,ТОЩАЯ)|(ЖТ)|(ЖТ Э)|(ЖТ,)|(ЖТ3)|(ЖТМ)|(ЖТ ДЛ СХ)|(ЖТ БРАК)|(ЖТ/АБС)','(БЕЗЖТ)|(БЕЗ ЖТ)', '(АБС)|(БРАК АБС)|(БР МАТКА)|(БРАКАБС)|(АБ МАТКИ)', '(ПОЛИКИСТ)|(ПОЛИКИС)', '(ДЛ СХ)|(ДЛСХ)|(ДЛ СХЕМА)|(ДЛСХ ОХО)', '(ГПФ)|(ГИПОФУНК)|(ГИПОФ)', '(БРАК)|(ХУДАЯ)|(БРАКНОГИ)|(БРАКТОЩА)|(БРАКЖИРН)|(БР ЖИРНА)|(ИЗ БРАКА)', '(ФОЛ)|(ФЛ)']
RemarkDRY   = ['(CEBA)|(СЕВА)|(СEBA)']
RemarkBROKE = ['(ATP4)|(АТР4)', '(2 СОСКА)|(2СОСКА)']


def gr(l):
    return list(map(lambda x: EventParsingRule(x, lambda t: 1), l))
    
EventRules = {
    EventType.OTHER   : [RuleBin],
    EventType.ABORT   : generate_combined_rules([RuleDays] + gr(RemarkABORT), EventType.ABORT),
    EventType.TOSCM   : generate_unique_rules(EventType.TOSCM), 
    EventType.NULSCM  : [RuleBin],
    EventType.DNB     : generate_combined_rules(gr(RemarkDNB), EventType.DNB), 
    EventType.BRED    : generate_unique_rules(EventType.BRED), 
    EventType.FRESH   : [RuleBin],  
    EventType.PREG    : [RuleDays], 
    EventType.PREGBEF : [RuleDays],
    EventType.DRY     : generate_combined_rules(gr(RemarkDRY), EventType.DRY),
    EventType.DRY2    : [RuleBin],
    EventType.OPEN    : generate_combined_rules(gr(RemarkOPEN), EventType.OPEN), 
    #-------
    EventType.WEIGHT  : [RuleNum],
    EventType.DEAD    : generate_unique_rules(EventType.DEAD),
    EventType.MOVE    : [RuleBin],
    EventType.SOLD    : generate_unique_rules(EventType.SOLD),
    #-------
    EventType.VAC     : generate_unique_rules(EventType.VAC),
    EventType.VACVIR  : generate_unique_rules(EventType.VACVIR),
    EventType.FOOTRIM : generate_unique_rules(EventType.FOOTRIM), # Может быть тут Bin поставить?
    EventType.HEALTH  : [RuleBin],
    EventType.BROKE   : generate_combined_rules(gr(RemarkBROKE), EventType.BROKE),
    EventType.POT     : generate_unique_rules(EventType.POT),
    EventType.ILLMISC : generate_unique_rules(EventType.ILLMISC),
    EventType.LAME    : generate_protocol_rules(EventType.LAME),
    EventType.KETOS   : generate_unique_rules(EventType.KETOS),
    EventType.MAST    : generate_protocol_rules(EventType.MAST),
    EventType.METR    : generate_unique_rules(EventType.METR),
    EventType.PARES   : generate_unique_rules(EventType.PARES),
    EventType.RP      : generate_unique_rules(EventType.RP),
}

#Get All Rules:
print("Правила парсинга:")
for event in EventType:
    print(str(event)+':')
    for number in range(len(EventRules[event])):
        print(str(event)+'_'+str(number) + ' :', EventRules[event][number].raw)
    print('------')

Правила парсинга:
OTHER:
OTHER_0 : .*
------
ABORT:
ABORT_0 : [0-9]* ((ДНИ)|(DAYS))
ABORT_1 : (БЖТ)|(БЕЗ ЖТ)
ABORT_2 : (ЖТ)|(ЖТ,)
ABORT_3 : (ДЛ СХЕМА)|(ДЛ СХ)|(ДС)
ABORT_4 : АБОРТ
ABORT_5 : ОХОТА
ABORT_6 : ФК
ABORT_7 : 0
ABORT_8 : ЛК
ABORT_9 : -
ABORT_10 : ЖИРНАЯ
ABORT_11 : БРАК
ABORT_12 : БРАКГИНЕ
ABORT_13 : БЫЛОСЕМ
ABORT_14 : БРАКЖИРН
------
TOSCM:
TOSCM_0 : 5
TOSCM_1 : 9
TOSCM_2 : 11
TOSCM_3 : 2
TOSCM_4 : 13
TOSCM_5 : 14
TOSCM_6 : 15
TOSCM_7 : 16
TOSCM_8 : 40
TOSCM_9 : 31
TOSCM_10 : 29
TOSCM_11 : 30
TOSCM_12 : 38
TOSCM_13 : 33
TOSCM_14 : 46
TOSCM_15 : 34
TOSCM_16 : 36
TOSCM_17 : 55
TOSCM_18 : 60
TOSCM_19 : 53
TOSCM_20 : 37
TOSCM_21 : 52
TOSCM_22 : 42
TOSCM_23 : 43
TOSCM_24 : 58
TOSCM_25 : 47
TOSCM_26 : 48
TOSCM_27 : 41
TOSCM_28 : 49
------
NULSCM:
NULSCM_0 : .*
------
DNB:
DNB_0 : (ЗООТЕХ)|(ЗООТ)|(ЗООТ3)|(ЗООБРАК)|(ЗОOТЕХ)
DNB_1 : (СУСТАВ)|(СУСТАВЫ)|(СУСТ_ВЫМ)
DNB_2 : (ГИНЕКОЛ)|(ГЕНИКОЛ)|(ГЕНЕКОЛ)
DNB_3 : (АТП)|(АТ)
DNB_4 : (AM)|(АМ)|(АВС)|(АБСМАТКИ)|(АБС МАТК)|(МАТКА)|(Б/МАТКИ)
DNB

Посмотрим что получилось:

In [ ]:
def Parseinfo(event, remark):
    global EventRules
    for rule in EventRules[event]:
        if rule.pattern.match(remark):
            return (rule.parse(remark))
    return 0

for event in EventType:
    df[str(event)] =  df.apply(lambda row: 1 if row.event == event else 0, axis=1)
    for number in range(len(EventRules[event])):
        df[str(event)+'_'+str(number)] = df.apply(lambda row: Parseinfo(event, row.remark) if row.event == event else 0, axis=1)


In [ ]:
df.head()

## Учёт истории и переводы
Теперь мы переходим к обработке контексто-зависимых забытий. Для этого надо иметь понимаение какое событие произошло ДО текущего события и ПОСЛЕ текущего события.

Как понять очередность событий, если они происходят в один день? Для начала надо посмотреть на примеры такого.

In [161]:
sf = df.groupby(['id', 'date']).count()
sf[sf['event'] > 3]

,,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,remark
id,date,,,,,,,,,,,,
4146,7/3/2022,4,4,4,4,4,4,4,4,4,4,4,4
4245,7/29/2022,4,4,4,4,4,4,4,4,4,4,4,4
6214,7/1/2022,4,4,4,4,4,4,4,4,4,4,4,4
8036,7/29/2022,4,4,4,4,4,4,4,4,4,4,4,4
11293,7/20/2022,4,4,0,4,4,4,4,4,4,4,4,4
12171,7/20/2022,4,4,0,4,4,4,4,4,4,4,4,4
20398,7/21/2022,4,4,0,4,4,4,4,4,4,4,4,4
20855,7/12/2022,4,4,0,4,4,4,4,4,4,4,4,4
803063,7/8/2022,4,4,4,4,4,4,4,4,4,4,4,4


Иногда (но очень редко), с коровой происходит несколько событий в один день. Как правило эти события отделены переводом, а между переводами порядок не важен.

Кстати параметр "Предыдущая группа животного" указывает на группу в предыдущем событии! Сортировка событий в течении дня происходит так:
1. Разбить события на на группы (каждая группа отвечает что делали с животным когда оно было в соотвествующей группе)
2. Упорядочить группы в соответсвии с информацией о перводах
3. Упорядочить события внутри группы по порядку из Enuma

### События перемещения:
Надо проверить, что события перемещения отображают корректную информацию

In [111]:
df['incorrect_move'] = df.apply(lambda row: 1 if row.event == EventType.MOVE and int(row.remark[1:4]) != row.group_id else 0, axis=1)
df[df['incorrect_move'] == 1].head()

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,incorrect_move


### Сортировка событий:

**TODO**:
Написано максимально криво надо подправить. К тому же есть вопросы к эффективности

In [9]:
print(df[df['event'] == EventType.FRESH]['group_id'].unique().tolist())

cond = ((df['event'] == EventType.FRESH) & (df['group_id'] == 18))
df[cond].head(10)

[18, 17, 45, 33, 40]


,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index
126,8136,8136,F,1,FA,1,8/17/2020,0,0,18,8,106,0,7/1/2022,-,127
353,612062,612062,F,4,MA,1,12/12/2016,63,0,18,21,106,0,7/1/2022,-,354
432,811180,811180,F,3,MA,1,11/24/2018,57,0,18,10,106,0,7/1/2022,-,433
475,907168,907168,F,2,FA,1,7/24/2019,64,0,18,4,106,0,7/1/2022,-,476
480,907225,907225,F,2,FA,1,7/30/2019,71,0,18,10,106,0,7/1/2022,-,481
487,908049,908049,F,2,MA,1,8/9/2019,49,0,18,43,106,0,7/1/2022,-,488
502,909158,909158,F,2,FA,1,9/21/2019,57,0,18,10,106,0,7/1/2022,-,503
512,910131,910131,F,2,FA,1,10/19/2019,57,0,18,10,106,0,7/1/2022,-,513
514,910171,910171,F,2,FA,1,10/26/2019,64,0,18,10,106,0,7/1/2022,-,515
604,807075,807075,F,3,FA,1,7/14/2018,58,0,18,2,106,0,7/2/2022,-,605


In [12]:
import itertools
def path_sort(edge_list):
    m = len(edge_list)
    if m == 1:
        return edge_list
    
    for sigma in itertools.permutations(list(range(m))):
        bad=False
        for i in range(1, m):
            if edge_list[sigma[i]][1] != edge_list[sigma[i-1]][0]:
                bad=True
                break
        if not bad:
            edge_list_fixed = [edge_list[sigma[i]] for i in range(1, m)]
            return edge_list_fixed
    return edge_list


def get_day_ordering(r):
    
    cow_id   = r.id
    ev_date  = r.date 
    ev_index = r.event_index
    
    global df
    today = df[(df['id'] == cow_id) & (df['date'] == ev_date)].copy()
    
    moves = today[today['event'] == EventType.MOVE]
    if moves.shape[0] == 0:
        today.sort_values(by=['event'])
        today['ord'] = list(range(1, today.shape[0] + 1, 1))
        return (today[today['event_index'] == ev_index]['ord'].iloc[0])
   
    moves['to']   = moves['remark'].apply(lambda x: int(x[5::]))
    moves['from'] = moves['remark'].apply(lambda x: int(x[1:4:]))
    
    edge_list  = path_sort(moves.apply(lambda row: (row['from'], row['to']), axis=1).tolist())
    path = [edge[0] for edge in edge_list]
    path.append(edge_list[-1][1])
    
    try:
        today['group_ord'] = today['group_id'].apply(lambda x: path.index(x))
        today.sort_values(by=['group_ord', 'event'])
    except:
        #print('------------------')
        #print(edge_list)
        #print(row)
        #print('------------------')
        today.sort_values(by=['event'])
        
    today['ord'] = list(range(1, today.shape[0] + 1, 1))
    
    return (today[today['event_index'] == ev_index]['ord'].iloc[0])

In [13]:
df['ord'] = df.apply(lambda row: get_day_ordering(row), axis=1)
df.sort_values(by=['id', 'date', 'ord'])

------------------
[(44, 21)]
id                    8136
sid                   8136
sex                      F
cycle                    1
childs                  FA
childbirth               1
dob              8/17/2020
deadwood_days            0
pregancy_days            0
group_id                44
group_past               8
event                  303
milking_days             0
date              7/1/2022
remark            F044T021
event_index            126
ord                      1
Name: 125, dtype: object
------------------
------------------
[(44, 21)]
id                    8136
sid                   8136
sex                      F
cycle                    1
childs                  FA
childbirth               1
dob              8/17/2020
deadwood_days            0
pregancy_days            0
group_id                18
group_past               8
event                  106
milking_days             0
date              7/1/2022
remark                   -
event_index            127
ord  

------------------
[(5, 12)]
id                 702036
sid                702036
sex                     F
cycle                   4
childs                 FA
childbirth              1
dob              2/5/2017
deadwood_days          52
pregancy_days         158
group_id                1
group_past             12
event                 207
milking_days          205
date             7/5/2022
remark             СУСТАВ
event_index          1533
ord                     1
Name: 1532, dtype: object
------------------
------------------
[(5, 12)]
id                 702036
sid                702036
sex                     F
cycle                   4
childs                 FA
childbirth              1
dob              2/5/2017
deadwood_days          52
pregancy_days         158
group_id               12
group_past             12
event                 204
milking_days          205
date             7/5/2022
remark            ХРОМОТА
event_index          1534
ord                     2
Name: 1533, d

------------------
[(17, 45)]
id                  709076
sid                 709076
sex                      F
cycle                    3
childs                  MA
childbirth               1
dob              9/16/2017
deadwood_days           63
pregancy_days          215
group_id                42
group_past              17
event                  109
milking_days           343
date              7/6/2022
remark                   -
event_index           1869
ord                      1
Name: 1868, dtype: object
------------------
------------------
[(17, 45)]
id                  709076
sid                 709076
sex                      F
cycle                    3
childs                  MA
childbirth               1
dob              9/16/2017
deadwood_days           63
pregancy_days          215
group_id                17
group_past              17
event                  303
milking_days           343
date              7/6/2022
remark            F017T045
event_index           1870
ord 

------------------
[(6, 12)]
id                    8036
sid                   8036
sex                      F
cycle                    1
childs                  FD
childbirth               2
dob               8/6/2020
deadwood_days            0
pregancy_days            0
group_id                 6
group_past               8
event                  105
milking_days            59
date             7/29/2022
remark           614H15481
event_index           4489
ord                      1
Name: 4488, dtype: object
------------------
------------------
[(6, 12)]
id                    8036
sid                   8036
sex                      F
cycle                    1
childs                  FD
childbirth               2
dob               8/6/2020
deadwood_days            0
pregancy_days            0
group_id                 9
group_past               8
event                  102
milking_days            59
date             7/29/2022
remark                  13
event_index           4490
ord   

------------------
[(44, 21)]
id                  606021
sid                 606021
sex                      F
cycle                    5
childs                  MA
childbirth               1
dob               6/4/2016
deadwood_days           57
pregancy_days            0
group_id                18
group_past              21
event                  106
milking_days             0
date             7/22/2022
remark                   -
event_index           9103
ord                      1
Name: 9102, dtype: object
------------------
------------------
[(44, 21)]
id                  606021
sid                 606021
sex                      F
cycle                    5
childs                  MA
childbirth               1
dob               6/4/2016
deadwood_days           57
pregancy_days            0
group_id                44
group_past              21
event                  213
milking_days             0
date             7/22/2022
remark             ПОСЛЕД2
event_index           9104
ord 

------------------
[(6, 12)]
id                  811042
sid                 811042
sex                      F
cycle                    2
childs                  FA
childbirth               1
dob              11/6/2018
deadwood_days           66
pregancy_days          230
group_id                 6
group_past               7
event                  303
milking_days           289
date             7/19/2022
remark            F006T012
event_index          10044
ord                      1
Name: 10043, dtype: object
------------------
------------------
[(6, 12)]
id                  811042
sid                 811042
sex                      F
cycle                    2
childs                  FA
childbirth               1
dob              11/6/2018
deadwood_days           66
pregancy_days          230
group_id                12
group_past               7
event                  204
milking_days           289
date             7/19/2022
remark             ХРОМОТА
event_index          10045
ord  

------------------
[(44, 21)]
id                  910033
sid                 910033
sex                      F
cycle                    2
childs                  FA
childbirth               1
dob              10/5/2019
deadwood_days           56
pregancy_days            0
group_id                44
group_past              21
event                  303
milking_days             0
date             7/28/2022
remark            F044T021
event_index          11055
ord                      1
Name: 11054, dtype: object
------------------
------------------
[(44, 21)]
id                  910033
sid                 910033
sex                      F
cycle                    2
childs                  FA
childbirth               1
dob              10/5/2019
deadwood_days           56
pregancy_days            0
group_id                44
group_past              21
event                  206
milking_days             0
date             7/28/2022
remark               ПРОФ2
event_index          11056
ord

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index,ord
1965,1001,1001,F,1,FA,1,1/1/2020,0,203,9,9,203,248,7/7/2022,КАН,1966,1
3805,1005,1005,F,1,MA,1,1/1/2020,0,173,6,6,303,254,7/13/2022,F006T007,3806,1
2881,1007,1007,F,1,FA,1,1/1/2020,0,256,5,17,303,268,7/11/2022,F005T006,2882,1
3806,1007,1007,F,1,FA,1,1/1/2020,0,258,6,17,107,275,7/18/2022,214 ДНИ,3807,1
3807,1007,1007,F,1,FA,1,1/1/2020,0,258,6,17,109,278,7/21/2022,CEBA,3808,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11554,912269,912269,F,1,FA,1,12/31/2019,0,223,7,7,303,242,7/19/2022,F007T012,11555,1
11555,912269,912269,F,1,FA,1,12/31/2019,0,223,12,7,204,242,7/19/2022,ХРОМОТА,11556,2
11556,912270,912270,F,1,MA,1,12/31/2019,0,257,7,17,203,275,7/16/2022,КАН,11557,1
11557,912270,912270,F,1,MA,1,12/31/2019,0,257,7,17,107,277,7/18/2022,213 ДНИ,11558,1


**TODO** Спросить что вообще с номерами групп происходит:

In [15]:
df[((df['date'] == '7/29/2022') & (df['id'] == 8036))].head(10)

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index,ord
4488,8036,8036,F,1,FD,2,8/6/2020,0,0,6,8,105,59,7/29/2022,614H15481,4489,1
4489,8036,8036,F,1,FD,2,8/6/2020,0,0,9,8,102,59,7/29/2022,13,4490,2
4490,8036,8036,F,1,FD,2,8/6/2020,0,0,6,8,303,59,7/29/2022,F006T012,4491,3
4491,8036,8036,F,1,FD,2,8/6/2020,0,0,12,8,204,59,7/29/2022,ХРОМОТА,4492,4


In [16]:
df[((df['date'] == '7/22/2022') & (df['id'] == 807163))].head(10)

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index,ord
9797,807163,807163,F,3,FA,1,7/28/2018,106,0,44,2,303,0,7/22/2022,F044T021,9798,1
9798,807163,807163,F,3,FA,1,7/28/2018,106,0,18,2,106,0,7/22/2022,-,9799,2


На самом деле такое решение имеет смысл только если нет ситуации когда животное кидают из группы в группу по циклу (например перевели из 3 в 12 потом обратно в 3), в таком случае восстановить порядок наверняка невозможно. Но судя по всему такого не происходит

###  Cобытия выздоровления и болезней:
Заболевания КЕТОЗ, МЕТРИТ, ПАРЕЗ, ПОСЛЕД, ПРОФОТ протекают в новотельный период и имеют общий критерий выписки: все послеотельные осложнения животного вылечили, корова пригодна к дальнейшей работе и воспроизводству (примечание при выписке ГИНЕКОЛ). Примечание МОЛОДНЯК к событию ЗДОРОВА имеют только животные 0-й лактации (телки и бычки). Примечание ПРОЧИЕ к событию ЗДОРОВА имеют животные, перенесшие прочие заболевания (событие БОЛЕЗНЬ).

In [42]:
df.head()

,id,sex,cycle,childs,childbirth,deadwood_days,pregancy_days,group_id,group_past,event,...,MAST_КМ6_3,MAST_КМ6_4,MAST_ТМ1_1,MAST_ТМ1_2,MAST_ТМ1_3,MAST_ТМ1_4,MAST_КМ3_1,MAST_КМ3_2,MAST_КМ3_3,MAST_КМ3_4
0,1064,1,1,FA,1,0,0,9,9,BRED,...,0,0,0,0,0,0,0,0,0,0
1,1064,1,1,FA,1,0,0,9,9,TOSCM,...,0,0,0,0,0,0,0,0,0,0
2,1185,1,1,FA,1,0,49,9,43,BRED,...,0,0,0,0,0,0,0,0,0,0
3,1185,1,1,FA,1,0,49,9,43,TOSCM,...,0,0,0,0,0,0,0,0,0,0
4,1260,1,1,FA,1,0,49,5,5,TOSCM,...,0,0,0,0,0,0,0,0,0,0


# Генерация новых фичей